# Long-Term Memory Experiments

This notebook explores long-term memory functionality, pattern learning, and knowledge management for AI agents.

In [ ]:
import sys
from pathlib import Path
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent.parent / "backend"))

from src.job_automation.core.memory import LongTermMemory, MemoryManager

print("Long-term memory environment ready!")

## Knowledge Base Management

In [ ]:
# Create long-term memory instance
ltm = LongTermMemory()

# Add job search knowledge
ltm.add_knowledge(
    "job_search_strategies",
    "effective_application_practices",
    {
        "personalize_applications": True,
        "research_company_culture": True,
        "tailor_resume_to_job": True,
        "follow_up_timeline": "1-2 weeks",
        "success_rate_improvement": "40-60%"
    }
)

# Add company-specific knowledge
companies_data = {
    "google": {
        "culture": "innovation-focused",
        "interview_style": "technical + behavioral",
        "hiring_timeline": "4-8 weeks",
        "remote_policy": "hybrid"
    },
    "netflix": {
        "culture": "high-performance",
        "interview_style": "culture-fit heavy",
        "hiring_timeline": "3-6 weeks",
        "remote_policy": "flexible"
    },
    "startups": {
        "culture": "fast-paced",
        "interview_style": "practical + speed",
        "hiring_timeline": "1-3 weeks",
        "remote_policy": "varies"
    }
}

for company, data in companies_data.items():
    ltm.add_knowledge(
        "companies",
        company,
        data,
        metadata={"category": "tech_companies", "verified": True}
    )

print(f"Added knowledge for {len(companies_data)} companies")
print(f"Knowledge categories: {list(ltm.knowledge_base.keys())}")

In [ ]:
# Retrieve and display knowledge
job_strategies = ltm.get_knowledge("job_search_strategies", "effective_application_practices")
print("Job Search Strategies:")
print(json.dumps(job_strategies, indent=2))

print("\nCompany Information:")
google_info = ltm.get_knowledge("companies", "google")
print(f"Google: {json.dumps(google_info, indent=2)}")

# Get all companies
all_companies = ltm.get_knowledge("companies")
print(f"\nTotal companies in knowledge base: {len(all_companies)}")

## Experience Tracking and Pattern Learning

In [ ]:
# Add successful job application experiences
successful_experiences = [
    {
        "personalized_cover_letter": True,
        "researched_company": True,
        "applied_within_24h": True,
        "followed_up": True,
        "salary_range_match": True
    },
    {
        "personalized_cover_letter": True,
        "researched_company": True,
        "applied_within_24h": False,
        "followed_up": True,
        "salary_range_match": True
    },
    {
        "personalized_cover_letter": True,
        "researched_company": False,
        "applied_within_24h": True,
        "followed_up": True,
        "salary_range_match": True
    }
]

for exp_data in successful_experiences:
    ltm.add_experience("job_application", exp_data, "success")

# Add some failed experiences
failed_experiences = [
    {
        "personalized_cover_letter": False,
        "researched_company": False,
        "applied_within_24h": False,
        "followed_up": False,
        "salary_range_match": False
    },
    {
        "personalized_cover_letter": False,
        "researched_company": True,
        "applied_within_24h": False,
        "followed_up": False,
        "salary_range_match": True
    }
]

for exp_data in failed_experiences:
    ltm.add_experience("job_application", exp_data, "failure")

print(f"Added {len(successful_experiences)} successful and {len(failed_experiences)} failed experiences")
print(f"Total experiences: {len(ltm.experiences)}")

In [ ]:
# Analyze learned patterns
patterns = ltm.get_patterns("job_application")
print("Learned Patterns:")
print(json.dumps(patterns, indent=2))

# Visualize pattern frequencies
if patterns and "success_patterns" in patterns:
    success_frequencies = [p["frequency"] for p in patterns["success_patterns"]]
    failure_frequencies = [p["frequency"] for p in patterns.get("failure_patterns", [])]
    
    plt.figure(figsize=(10, 6))
    
    plt.subplot(1, 2, 1)
    if success_frequencies:
        plt.hist(success_frequencies, bins=5, alpha=0.7, color='green')
        plt.title('Success Pattern Frequencies')
        plt.xlabel('Frequency')
        plt.ylabel('Count')
    
    plt.subplot(1, 2, 2)
    if failure_frequencies:
        plt.hist(failure_frequencies, bins=5, alpha=0.7, color='red')
        plt.title('Failure Pattern Frequencies')
        plt.xlabel('Frequency')
        plt.ylabel('Count')
    
    plt.tight_layout()
    plt.show()

## Experience Analysis and Insights

In [ ]:
# Convert experiences to DataFrame for analysis
experiences_df = pd.DataFrame(ltm.experiences)

if not experiences_df.empty:
    print("Experience Analysis:")
    print(f"Total experiences: {len(experiences_df)}")
    print(f"Success rate: {(experiences_df['outcome'] == 'success').mean():.2%}")
    
    # Analyze outcomes by type
    outcome_counts = experiences_df['outcome'].value_counts()
    print(f"\nOutcome distribution:")
    for outcome, count in outcome_counts.items():
        print(f"  {outcome}: {count}")
    
    # Plot outcome distribution
    plt.figure(figsize=(8, 6))
    outcome_counts.plot(kind='pie', autopct='%1.1f%%')
    plt.title('Experience Outcomes Distribution')
    plt.ylabel('')
    plt.show()
    
    # Analyze experience data features
    # Extract features from the data column
    features_data = []
    for _, row in experiences_df.iterrows():
        feature_row = row['data'].copy()
        feature_row['outcome'] = row['outcome']
        features_data.append(feature_row)
    
    features_df = pd.DataFrame(features_data)
    
    # Calculate success rates by feature
    print("\nSuccess rates by feature:")
    for column in features_df.columns:
        if column != 'outcome' and features_df[column].dtype == bool:
            success_rate = features_df[features_df[column] == True]['outcome'].apply(lambda x: x == 'success').mean()
            print(f"  {column}: {success_rate:.2%}")
else:
    print("No experiences to analyze")

## Feature Correlation Analysis

In [ ]:
if not experiences_df.empty and 'features_df' in locals():
    # Convert boolean features to numeric for correlation analysis
    numeric_features = features_df.copy()
    for col in numeric_features.columns:
        if numeric_features[col].dtype == bool:
            numeric_features[col] = numeric_features[col].astype(int)
    
    # Convert outcome to numeric (1 for success, 0 for failure)
    numeric_features['outcome_numeric'] = (numeric_features['outcome'] == 'success').astype(int)
    
    # Calculate correlation matrix
    correlation_matrix = numeric_features.drop('outcome', axis=1).corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f')
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Show correlations with success outcome
    outcome_correlations = correlation_matrix['outcome_numeric'].drop('outcome_numeric').sort_values(ascending=False)
    print("Correlations with success outcome:")
    for feature, corr in outcome_correlations.items():
        print(f"  {feature}: {corr:.3f}")

## Memory Manager Integration Testing

In [ ]:
# Test full memory manager with both short and long-term memory
manager = MemoryManager()

# Add knowledge through manager
manager.add_knowledge(
    "interview_preparation",
    "technical_interview_tips",
    {
        "practice_coding_problems": True,
        "understand_system_design": True,
        "prepare_behavioral_examples": True,
        "research_interviewer_background": True,
        "preparation_time_recommended": "2-4 weeks"
    }
)

# Add experiences through manager
interview_experiences = [
    {
        "prepared_coding_problems": True,
        "researched_company": True,
        "mock_interviews": 3,
        "prepared_questions": True
    },
    {
        "prepared_coding_problems": False,
        "researched_company": True,
        "mock_interviews": 0,
        "prepared_questions": False
    }
]

manager.add_experience("interview", interview_experiences[0], "success")
manager.add_experience("interview", interview_experiences[1], "failure")

# Get comprehensive memory stats
stats = manager.get_memory_stats()
print("Comprehensive Memory Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

# Test knowledge retrieval
interview_tips = manager.get_knowledge("interview_preparation", "technical_interview_tips")
print("\nInterview Tips from Knowledge Base:")
print(json.dumps(interview_tips, indent=2))

# Test experience retrieval
interview_experiences = manager.get_experiences("interview")
print(f"\nInterview Experiences: {len(interview_experiences)} recorded")

# Test pattern learning
interview_patterns = manager.get_patterns("interview")
print("\nLearned Interview Patterns:")
if interview_patterns:
    print(json.dumps(interview_patterns, indent=2))
else:
    print("No patterns learned yet (need more data)")

## Save Experiment Results

In [ ]:
# Compile experiment results
experiment_results = {
    "experiment_date": datetime.now().isoformat(),
    "knowledge_base_size": len(ltm.knowledge_base),
    "total_experiences": len(ltm.experiences),
    "patterns_learned": len(ltm.patterns),
    "success_rate": (experiences_df['outcome'] == 'success').mean() if not experiences_df.empty else 0,
    "memory_manager_stats": stats
}

# Add pattern analysis if available
if 'outcome_correlations' in locals():
    experiment_results["feature_correlations"] = outcome_correlations.to_dict()

# Save results
output_dir = Path.cwd().parent / "experiments"
output_dir.mkdir(exist_ok=True)

with open(output_dir / "long_term_memory_experiment.json", "w") as f:
    json.dump(experiment_results, f, indent=2)

print(f"Experiment results saved to {output_dir / 'long_term_memory_experiment.json'}")

# Also save the knowledge base for future use
knowledge_export = {
    "knowledge_base": ltm.knowledge_base,
    "export_date": datetime.now().isoformat()
}

with open(output_dir / "knowledge_base_export.json", "w") as f:
    json.dump(knowledge_export, f, indent=2)

print(f"Knowledge base exported to {output_dir / 'knowledge_base_export.json'}")